# Module E: LoRA/QLoRA Fine-tuning Lab (Text-to-SQL)

**Goal:** Fine-tune a small instruction model to translate natural-language credit/risk questions into **SQLite SQL** using **QLoRA** (4-bit base + LoRA adapters).

**Persona:** FICO Cloud Engineer (cost/latency/privacy + deployment mindset)

## What you'll build
- A synthetic, FICO-flavored SQLite database (`customers`, `accounts`, `transactions`, `credit_applications`)
- A labeled Text-to-SQL dataset that is **validated by execution**
- A baseline model evaluation (exact match + execution match)
- A short but real **QLoRA training run** that saves adapter weights
- A post-tune evaluation + deployment-minded adapter loading (optional merge)

---

## Why parameter-efficient fine-tuning (PEFT)?
Full fine-tuning updates *all* model weights — expensive and operationally heavy.

**LoRA** updates only small low-rank matrices injected into key linear layers.

**QLoRA** combines:
- **4-bit** quantized *frozen* base model weights (cheap to store + load)
- **LoRA adapters** (small, trainable, easy to version and ship)

This is a practical deployment pattern:
- Host one base model once
- Load per-tenant/per-task adapters on demand

---

## Sections
- Part 0 — Setup + constraints
- Part 1 — What "modern fine-tuning" means (PEFT only)
- Part 2 — Synthetic Text-to-SQL dataset + sqlite ground truth
- Part 3 — Baseline inference (before tuning)
- Part 4 — QLoRA training (the main event)
- Part 5 — Evaluation after tuning
- Part 6 — Deployment mindset (adapter-only)

---

# Part 0 — Setup + Constraints

We'll verify GPU availability, import libraries, and set a deterministic seed.

In [45]:
# ============================================================
# Environment check: GPU / VRAM
# ============================================================
import sys
import os
import warnings
warnings.filterwarnings("ignore")

print("Python:", sys.executable)
print("Working dir:", os.getcwd())

# GPU check
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✅ GPU: {gpu_name} ({vram_gb:.1f} GB VRAM)")
        DEVICE = "cuda"
    else:
        print("⚠️  No CUDA GPU detected — training will be slow (CPU mode)")
        DEVICE = "cpu"
except ImportError:
    print("❌ PyTorch not installed")
    DEVICE = "cpu"

print(f"\nUsing device: {DEVICE}")

Python: /home/shadeform/workshop-v1/fico/.venv/bin/python
Working dir: /home/shadeform/workshop-v1/fico
✅ GPU: NVIDIA H200 (150.0 GB VRAM)

Using device: cuda


In [46]:
# ============================================================
# Imports
# ============================================================
import random
import sqlite3
import re
import time
import json
from pathlib import Path
from typing import List, Dict, Any, Tuple

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel,
)

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output

print("✅ All imports successful")

✅ All imports successful


In [47]:
# ============================================================
# Reproducibility: set seeds
# ============================================================
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"✅ Random seed set to {SEED}")

✅ Random seed set to 42


---

# Part 1 — What "Modern Fine-tuning" Means (PEFT Only)

We will **not** do full fine-tuning. Instead we use **LoRA / QLoRA**.

## Key Idea
```
┌──────────────────────────────────────────────────────────────────┐
│  Frozen 4-bit Base Weights   │  Trainable LoRA Adapters (r×d)   │
│  ~1.5 B params (quantized)   │  ~1-5 M params (fp16/bf16)       │
└──────────────────────────────────────────────────────────────────┘
```

**What gets trained?**
- Two small matrices **A** (d × r) and **B** (r × d) per target linear layer
- Effective weight update: `ΔW = B @ A` (low-rank)

**What stays frozen?**
- The original model weights (stored in 4-bit NF4)

**Benefits:**
| Aspect | Full Fine-tune | QLoRA |
|--------|----------------|-------|
| Trainable params | 100% | ~0.1-1% |
| GPU VRAM | 24-80 GB | 6-16 GB |
| Artifact size | Full model | Small adapter |
| Multi-tenant | Hard | Easy (swap adapters) |

In [48]:
# Quick demo: what does a LoRA config look like?
demo_lora_config = LoraConfig(
    r=16,                          # rank of the low-rank matrices
    lora_alpha=32,                 # scaling factor (alpha / r)
    lora_dropout=0.05,             # dropout on adapter activations
    target_modules=["q_proj", "v_proj"],  # which layers to adapt
    bias="none",
    task_type="CAUSAL_LM",
)
print("Example LoraConfig:")
print(demo_lora_config)

Example LoraConfig:
LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, peft_version='0.18.0', base_model_name_or_path=None, revision=None, inference_mode=False, r=16, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None, arrow_config=None, ensure_weight_tying=False)


---

# Part 2 — Synthetic Text-to-SQL Dataset + SQLite Ground Truth

We'll create a small FICO-flavored schema in **SQLite (in-memory)**, populate it with synthetic data, then generate a labeled dataset of (NL question, gold SQL) pairs.

## Schema
- `customers` — customer_id, name, email, fico_score, created_at
- `accounts` — account_id, customer_id, account_type, balance, opened_at
- `transactions` — txn_id, account_id, amount, txn_type, txn_date
- `credit_applications` — app_id, customer_id, requested_amount, status, decision_date

In [49]:
# ============================================================
# Create the SQLite database + schema
# ============================================================

DB_PATH = ":memory:"  # in-memory; change to file path if you want persistence

def get_connection():
    """Return a fresh connection to the database."""
    return sqlite3.connect(DB_PATH, check_same_thread=False)

def init_schema(conn: sqlite3.Connection):
    """Create tables."""
    cur = conn.cursor()
    cur.executescript("""
        DROP TABLE IF EXISTS customers;
        DROP TABLE IF EXISTS accounts;
        DROP TABLE IF EXISTS transactions;
        DROP TABLE IF EXISTS credit_applications;

        CREATE TABLE customers (
            customer_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            email TEXT,
            fico_score INTEGER,
            created_at TEXT
        );

        CREATE TABLE accounts (
            account_id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            account_type TEXT,   -- 'checking', 'savings', 'credit'
            balance REAL,
            opened_at TEXT,
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
        );

        CREATE TABLE transactions (
            txn_id INTEGER PRIMARY KEY,
            account_id INTEGER,
            amount REAL,
            txn_type TEXT,       -- 'credit', 'debit'
            txn_date TEXT,
            FOREIGN KEY (account_id) REFERENCES accounts(account_id)
        );

        CREATE TABLE credit_applications (
            app_id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            requested_amount REAL,
            status TEXT,         -- 'approved', 'denied', 'pending'
            decision_date TEXT,
            FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
        );
    """)
    conn.commit()

# Initialize
conn = get_connection()
init_schema(conn)
print("✅ Schema created")

✅ Schema created


In [50]:
# ============================================================
# Populate with synthetic data
# ============================================================
from datetime import datetime, timedelta

FIRST_NAMES = ["Alice", "Bob", "Carol", "David", "Eva", "Frank", "Grace", "Henry", "Ivy", "Jack"]
LAST_NAMES = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller", "Davis", "Martinez", "Wilson"]
ACCOUNT_TYPES = ["checking", "savings", "credit"]
TXN_TYPES = ["credit", "debit"]
APP_STATUSES = ["approved", "denied", "pending"]

def random_date(start_year=2020, end_year=2024):
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)
    delta = (end - start).days
    return (start + timedelta(days=random.randint(0, delta))).strftime("%Y-%m-%d")

def populate_data(conn, n_customers=50):
    cur = conn.cursor()
    for cid in range(1, n_customers + 1):
        name = f"{random.choice(FIRST_NAMES)} {random.choice(LAST_NAMES)}"
        email = f"user{cid}@example.com"
        fico = random.randint(300, 850)
        created = random_date(2018, 2022)
        cur.execute("INSERT INTO customers VALUES (?, ?, ?, ?, ?)", (cid, name, email, fico, created))
    
    aid = 1
    for cid in range(1, n_customers + 1):
        for _ in range(random.randint(1, 3)):
            atype = random.choice(ACCOUNT_TYPES)
            balance = round(random.uniform(-5000, 50000), 2)
            opened = random_date(2019, 2023)
            cur.execute("INSERT INTO accounts VALUES (?, ?, ?, ?, ?)", (aid, cid, atype, balance, opened))
            aid += 1
    
    tid = 1
    cur.execute("SELECT account_id FROM accounts")
    account_ids = [r[0] for r in cur.fetchall()]
    for acc_id in random.sample(account_ids, min(len(account_ids), 60)):
        for _ in range(random.randint(5, 20)):
            amt = round(random.uniform(5, 2000), 2)
            ttype = random.choice(TXN_TYPES)
            tdate = random_date(2022, 2024)
            cur.execute("INSERT INTO transactions VALUES (?, ?, ?, ?, ?)", (tid, acc_id, amt, ttype, tdate))
            tid += 1
    
    appid = 1
    for cid in random.sample(range(1, n_customers + 1), n_customers // 2):
        for _ in range(random.randint(1, 2)):
            req_amt = round(random.uniform(1000, 50000), 2)
            status = random.choice(APP_STATUSES)
            dec_date = random_date(2023, 2024)
            cur.execute("INSERT INTO credit_applications VALUES (?, ?, ?, ?, ?)", (appid, cid, req_amt, status, dec_date))
            appid += 1
    conn.commit()
    return aid - 1, tid - 1, appid - 1

n_accounts, n_txns, n_apps = populate_data(conn, n_customers=50)
print(f"Data populated: 50 customers, {n_accounts} accounts, {n_txns} transactions, {n_apps} applications")

Data populated: 50 customers, 97 accounts, 689 transactions, 38 applications


In [51]:
# Quick sanity check
pd.read_sql("SELECT * FROM customers LIMIT 5", conn)

,customer_id,name,email,fico_score,created_at
0,1,Bob Smith,user1@example.com,581,2019-05-17
1,2,David Williams,user2@example.com,404,2021-10-17
2,3,Ivy Johnson,user3@example.com,732,2018-03-07
3,4,Alice Johnson,user4@example.com,523,2019-04-22
4,5,Ivy Wilson,user5@example.com,327,2021-02-23


In [52]:
# ============================================================
# Generate a labeled Text-to-SQL dataset
# ============================================================

SCHEMA_CONTEXT = """
Tables:
- customers(customer_id, name, email, fico_score, created_at)
- accounts(account_id, customer_id, account_type, balance, opened_at)
- transactions(txn_id, account_id, amount, txn_type, txn_date)
- credit_applications(app_id, customer_id, requested_amount, status, decision_date)
"""

# Templates: (instruction, sql, difficulty)
TEMPLATES = [
    # Simple (single table, no join)
    ("How many customers are there?", "SELECT COUNT(*) FROM customers;", "simple"),
    ("List all customer names.", "SELECT name FROM customers;", "simple"),
    ("What is the average FICO score?", "SELECT AVG(fico_score) FROM customers;", "simple"),
    ("Show customers with FICO score above 700.", "SELECT * FROM customers WHERE fico_score > 700;", "simple"),
    ("How many accounts are of type 'checking'?", "SELECT COUNT(*) FROM accounts WHERE account_type = 'checking';", "simple"),
    ("What is the total balance across all accounts?", "SELECT SUM(balance) FROM accounts;", "simple"),
    ("List all transactions over 500 dollars.", "SELECT * FROM transactions WHERE amount > 500;", "simple"),
    ("How many credit applications are pending?", "SELECT COUNT(*) FROM credit_applications WHERE status = 'pending';", "simple"),
    
    # Medium (single join or GROUP BY)
    ("Show the total balance per customer.", 
     "SELECT c.customer_id, c.name, SUM(a.balance) AS total_balance FROM customers c JOIN accounts a ON c.customer_id = a.customer_id GROUP BY c.customer_id;", 
     "medium"),
    ("How many accounts does each customer have?", 
     "SELECT c.customer_id, c.name, COUNT(a.account_id) AS num_accounts FROM customers c JOIN accounts a ON c.customer_id = a.customer_id GROUP BY c.customer_id;", 
     "medium"),
    ("What is the average transaction amount per account?", 
     "SELECT account_id, AVG(amount) AS avg_amount FROM transactions GROUP BY account_id;", 
     "medium"),
    ("List customers who have applied for credit.", 
     "SELECT DISTINCT c.customer_id, c.name FROM customers c JOIN credit_applications ca ON c.customer_id = ca.customer_id;", 
     "medium"),
    ("Show all approved credit applications with customer names.", 
     "SELECT c.name, ca.requested_amount, ca.decision_date FROM customers c JOIN credit_applications ca ON c.customer_id = ca.customer_id WHERE ca.status = 'approved';", 
     "medium"),
    
    # Complex (multi-join, subquery, or aggregation+filter)
    ("Find customers with FICO above 750 who have been denied credit.", 
     "SELECT c.customer_id, c.name, c.fico_score FROM customers c JOIN credit_applications ca ON c.customer_id = ca.customer_id WHERE c.fico_score > 750 AND ca.status = 'denied';", 
     "complex"),
    ("What is the total transaction amount for customers with savings accounts?", 
     "SELECT SUM(t.amount) FROM transactions t JOIN accounts a ON t.account_id = a.account_id WHERE a.account_type = 'savings';", 
     "complex"),
    ("Show the top 5 customers by total account balance.", 
     "SELECT c.customer_id, c.name, SUM(a.balance) AS total FROM customers c JOIN accounts a ON c.customer_id = a.customer_id GROUP BY c.customer_id ORDER BY total DESC LIMIT 5;", 
     "complex"),
    ("List customers who have more than 2 accounts.", 
     "SELECT c.customer_id, c.name, COUNT(a.account_id) AS cnt FROM customers c JOIN accounts a ON c.customer_id = a.customer_id GROUP BY c.customer_id HAVING cnt > 2;", 
     "complex"),
]

def validate_sql(conn, sql: str) -> Tuple[bool, Any]:
    """Execute SQL and return (success, result_or_error)."""
    try:
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        return True, rows
    except Exception as e:
        return False, str(e)

def build_dataset(conn, templates: List[Tuple[str, str, str]]) -> List[Dict]:
    """Build dataset from templates, validating each SQL."""
    dataset = []
    for instruction, sql, difficulty in templates:
        ok, result = validate_sql(conn, sql)
        if not ok:
            print(f"WARNING: SQL failed validation: {sql[:50]}... -> {result}")
            continue
        dataset.append({
            "instruction": instruction,
            "context": SCHEMA_CONTEXT.strip(),
            "response": sql.strip(),
            "difficulty": difficulty,
            "num_rows": len(result) if isinstance(result, list) else 0,
        })
    return dataset

text2sql_data = build_dataset(conn, TEMPLATES)
print(f"Dataset built: {len(text2sql_data)} examples")
print(f"  simple: {sum(1 for d in text2sql_data if d['difficulty']=='simple')}")
print(f"  medium: {sum(1 for d in text2sql_data if d['difficulty']=='medium')}")
print(f"  complex: {sum(1 for d in text2sql_data if d['difficulty']=='complex')}")

Dataset built: 17 examples
  simple: 8
  medium: 5
  complex: 4


### Alternative: Load Pre-Generated Dataset (Recommended)

Instead of using the small template-based dataset above, load a larger LLM-generated dataset from `text2sql_generated.json`.

This cell performs a **stratified train/test split** — ensuring each difficulty level is proportionally represented in both sets.

In [53]:
# ============================================================
# Load External Dataset + Stratified Train/Test Split
# ============================================================

import json
from pathlib import Path
from sklearn.model_selection import train_test_split
from collections import Counter

DATASET_PATH = Path("text2sql_generated.json")
TRAIN_RATIO = 0.9  # 90% train, 10% eval

if DATASET_PATH.exists():
    with open(DATASET_PATH) as f:
        all_examples = json.load(f)
    
    print(f"Loaded {len(all_examples)} examples from {DATASET_PATH}")
    
    # Show difficulty distribution
    difficulty_counts = Counter(ex["difficulty"] for ex in all_examples)
    print("\nDifficulty distribution:")
    for diff, count in sorted(difficulty_counts.items()):
        print(f"  {diff}: {count} ({100*count/len(all_examples):.1f}%)")
    
    # Stratified split by difficulty
    difficulties = [ex["difficulty"] for ex in all_examples]
    
    train_examples, eval_examples = train_test_split(
        all_examples,
        train_size=TRAIN_RATIO,
        stratify=difficulties,
        random_state=SEED
    )
    
    # Convert to the format expected by the rest of the notebook
    # Add schema context to each example
    SCHEMA_CONTEXT = """Tables:
- customers (customer_id, name, email, fico_score, created_at)
- accounts (account_id, customer_id, account_type, balance, opened_at)
- transactions (txn_id, account_id, amount, txn_type, txn_date)
- credit_applications (app_id, customer_id, requested_amount, status, decision_date)"""

    def convert_example(ex):
        return {
            "instruction": ex["instruction"],
            "context": SCHEMA_CONTEXT,
            "response": ex["sql"],
            "difficulty": ex["difficulty"]
        }
    
    text2sql_train = [convert_example(ex) for ex in train_examples]
    text2sql_eval = [convert_example(ex) for ex in eval_examples]
    
    # Also update text2sql_data for backward compatibility with other cells
    text2sql_data = text2sql_train + text2sql_eval
    
    # Show split statistics
    print(f"\n✅ Stratified split complete:")
    print(f"   Train: {len(text2sql_train)} examples")
    print(f"   Eval:  {len(text2sql_eval)} examples")
    
    # Verify stratification
    train_diffs = Counter(ex["difficulty"] for ex in text2sql_train)
    eval_diffs = Counter(ex["difficulty"] for ex in text2sql_eval)
    
    print("\n   Train distribution:")
    for diff in sorted(train_diffs.keys()):
        print(f"     {diff}: {train_diffs[diff]}")
    
    print("\n   Eval distribution:")
    for diff in sorted(eval_diffs.keys()):
        print(f"     {diff}: {eval_diffs[diff]}")
        
else:
    print(f"⚠️ Dataset file not found: {DATASET_PATH}")
    print("   Using the smaller template-based dataset from above.")
    print("   To use a larger dataset, generate one with the LLM prompt and save as text2sql_generated.json")

Loaded 221 examples from text2sql_generated.json

Difficulty distribution:
  complex: 60 (27.1%)
  medium: 66 (29.9%)
  simple: 95 (43.0%)

✅ Stratified split complete:
   Train: 198 examples
   Eval:  23 examples

   Train distribution:
     complex: 54
     medium: 59
     simple: 85

   Eval distribution:
     complex: 6
     medium: 7
     simple: 10


### Interactive: Explore the Dataset

Pick an example to see its schema, NL question, gold SQL, and a preview of the executed result.

In [54]:
# Interactive widget to explore dataset
w_example_idx = widgets.IntSlider(value=0, min=0, max=len(text2sql_data)-1, description="Example #")
w_output = widgets.Output()

def show_example(idx):
    with w_output:
        clear_output()
        ex = text2sql_data[idx]
        print("=" * 60)
        print(f"Difficulty: {ex['difficulty'].upper()}")
        print("=" * 60)
        print(f"\n[Question]\n{ex['instruction']}\n")
        print(f"[Schema Context]\n{ex['context']}\n")
        print(f"[Gold SQL]\n{ex['response']}\n")
        ok, result = validate_sql(conn, ex['response'])
        if ok:
            df = pd.DataFrame(result)
            print(f"[Result Preview] ({len(result)} rows)")
            display(df.head(5))
        else:
            print(f"[Execution Error] {result}")

widgets.interactive(show_example, idx=w_example_idx)
display(w_example_idx, w_output)
show_example(0)

IntSlider(value=0, description='Example #', max=220)

Output()

### Exercise A: Add Your Own Templates

Add **5 new templates** (mix of joins and aggregations) to `STUDENT_TEMPLATES`, then run the cell to regenerate and revalidate the dataset.

In [55]:
# TODO: Add 5 new templates (instruction, sql, difficulty)
STUDENT_TEMPLATES = [
    # Example (uncomment and modify):
    # ("What is the minimum FICO score?", "SELECT MIN(fico_score) FROM customers;", "simple"),
]

# Combine and rebuild
if STUDENT_TEMPLATES:
    combined = TEMPLATES + STUDENT_TEMPLATES
    text2sql_data = build_dataset(conn, combined)
    print(f"Dataset rebuilt with student templates: {len(text2sql_data)} examples")
else:
    print("No student templates added yet. Add yours above!")

No student templates added yet. Add yours above!


---

# Part 3 — Baseline Inference (Before Tuning)

We'll pick a small instruct model, define a prompt template, generate SQL for a few samples, then compute:
- **Exact match** (normalized SQL)
- **Execution match** (compare SQLite result sets)

> Default model: `Qwen/Qwen2.5-1.5B-Instruct` (upgrade to 3B if you have VRAM).

In [56]:
# ============================================================
# Prompt template + SQL normalization + execution-based eval
# ============================================================

def build_prompt(instruction: str, context: str) -> str:
    return (
        "You are a Text-to-SQL assistant for an SQLite database.\n"
        "Return ONLY a single SQL query, no explanation.\n\n"
        f"### Schema\n{context}\n\n"
        f"### Question\n{instruction}\n\n"
        "### SQL\n"
    )

_sql_ws = re.compile(r"\s+")

def normalize_sql(sql: str) -> str:
    # Keep this intentionally simple: no heavyweight parsers.
    s = sql.strip()
    # Keep only the first statement (best-effort)
    s = s.split(";")[0].strip()
    s = s.replace("\n", " ")
    s = _sql_ws.sub(" ", s)
    s = s.strip().lower()
    return s

def safe_exec(conn: sqlite3.Connection, sql: str):
    """Return (ok, rows_or_error). Rows are returned as tuples."""
    try:
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        return True, rows
    except Exception as e:
        return False, str(e)

def resultset_equal(a_rows, b_rows) -> bool:
    """Compare result sets ignoring row order (best-effort)."""
    try:
        a = sorted(list(a_rows))
        b = sorted(list(b_rows))
        return a == b
    except Exception:
        return False

def evaluate_predictions(conn, examples: List[Dict[str, Any]], preds: List[str]) -> pd.DataFrame:
    records = []
    for ex, pred in zip(examples, preds):
        gold_sql = ex["response"]
        pred_sql = pred

        gold_norm = normalize_sql(gold_sql)
        pred_norm = normalize_sql(pred_sql)
        exact = int(gold_norm == pred_norm)

        ok_g, rows_g = safe_exec(conn, gold_sql)
        ok_p, rows_p = safe_exec(conn, pred_sql)
        exec_match = int(ok_g and ok_p and resultset_equal(rows_g, rows_p))

        records.append({
            "difficulty": ex["difficulty"],
            "instruction": ex["instruction"],
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "exact_match": exact,
            "exec_match": exec_match,
            "pred_ok": bool(ok_p),
        })
    return pd.DataFrame(records)

print("✅ Helpers ready")

✅ Helpers ready


In [57]:
# ============================================================
# Load base model (baseline)
# ============================================================

BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"  # try "Qwen/Qwen2.5-3B-Instruct" if VRAM allows

# CPU-safe fallback (much smaller; for smoke tests only)
CPU_FALLBACK_MODEL = "sshleifer/tiny-gpt2"

use_model = BASE_MODEL

# Heuristic: if no GPU, default to tiny fallback so the notebook is runnable.
if DEVICE != "cuda":
    print("⚠️ CPU mode detected: using a tiny fallback model for demonstration.")
    use_model = CPU_FALLBACK_MODEL

print("Model:", use_model)

tokenizer = AutoTokenizer.from_pretrained(use_model, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Dtype/device_map
dtype = torch.bfloat16 if (DEVICE == "cuda" and torch.cuda.is_bf16_supported()) else (torch.float16 if DEVICE == "cuda" else torch.float32)

model = AutoModelForCausalLM.from_pretrained(
    use_model,
    torch_dtype=dtype,
    device_map="auto" if DEVICE == "cuda" else None,
)

if DEVICE != "cuda":
    model.to("cpu")

model.eval()
print("✅ Model loaded")

Model: Qwen/Qwen2.5-1.5B-Instruct


✅ Model loaded


In [58]:
# ============================================================
# Baseline generation on a handful of samples
# ============================================================

def generate_sql(model, tokenizer, instruction: str, context: str, max_new_tokens=128, temperature=0.0) -> str:
    prompt = build_prompt(instruction, context)
    inputs = tokenizer(prompt, return_tensors="pt")
    if DEVICE == "cuda":
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
    else:
        inputs = {k: v.to("cpu") for k, v in inputs.items()}

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature if temperature > 0 else None,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    t0 = time.time()
    with torch.no_grad():
        out = model.generate(**inputs, **{k: v for k, v in gen_kwargs.items() if v is not None})
    dt = time.time() - t0

    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # Extract everything after the prompt
    completion = text[len(prompt):].strip() if text.startswith(prompt) else text.strip()

    # Best-effort: take first line/statement
    completion = completion.split("\n")[0].strip()
    if ";" in completion:
        completion = completion.split(";")[0].strip() + ";"

    # latency estimate
    new_tokens = int(out.shape[-1] - inputs["input_ids"].shape[-1])
    tps = (new_tokens / dt) if dt > 0 and new_tokens > 0 else None
    return completion, {"seconds": dt, "new_tokens": new_tokens, "tokens_per_sec": tps}

# Use proper eval set if available, otherwise fallback to first N examples
if 'text2sql_eval' in dir() and text2sql_eval:
    EVAL_N = min(20, len(text2sql_eval))  # Use up to 20 eval examples
    examples_eval = text2sql_eval[:EVAL_N]
    print(f"Using {EVAL_N} examples from held-out eval set")
else:
    EVAL_N = 8
    examples_eval = text2sql_data[:EVAL_N]
    print(f"Using first {EVAL_N} examples (no separate eval set)")

preds = []
latencies = []
for ex in examples_eval:
    pred, lat = generate_sql(model, tokenizer, ex["instruction"], ex["context"], max_new_tokens=128)
    preds.append(pred)
    latencies.append(lat)

baseline_df = evaluate_predictions(conn, examples_eval, preds)
baseline_df["tokens_per_sec"] = [x.get("tokens_per_sec") for x in latencies]

baseline_df

Using 20 examples from held-out eval set


,difficulty,instruction,gold_sql,pred_sql,exact_match,exec_match,pred_ok,tokens_per_sec
0,medium,Show the latest transaction date for each acco...,"SELECT account_id, MAX(txn_date) FROM transact...",```sql,0,0,False,64.735038
1,complex,Show the monthly total transaction amount for ...,"SELECT strftime('%Y-%m', txn_date) as month, S...",SELECT SUM(amount) AS total FROM transactions ...,0,0,False,65.347432
2,medium,Show the number of distinct account types per ...,"SELECT c.name, COUNT(DISTINCT a.account_type) ...",```sql,0,0,False,65.466394
3,complex,List account types that have an average balanc...,SELECT account_type FROM accounts GROUP BY acc...,```sql,0,0,False,65.523266
4,simple,How many customers have a FICO score above 800?,SELECT COUNT(*) FROM customers WHERE fico_scor...,SELECT COUNT(DISTINCT c.customer_id) FROM cust...,0,1,True,65.720904
5,medium,Show the total approved credit amount per cust...,"SELECT c.name, SUM(ca.requested_amount) FROM c...",```sql,0,0,False,64.180101
6,simple,Show all customer names sorted alphabetically.,SELECT name FROM customers ORDER BY name ASC;,```sql,0,0,False,64.522349
7,medium,List account IDs and the number of credit tran...,"SELECT account_id, COUNT(*) FROM transactions ...",```sql,0,0,False,64.486837
8,simple,Retrieve the account ID for the account with t...,SELECT account_id FROM accounts ORDER BY balan...,SELECT account_id FROM accounts ORDER BY balan...,0,1,True,66.641242
9,complex,Find the maximum gap in days between transacti...,SELECT MAX(julianday(t1.txn_date) - julianday(...,```sql,0,0,False,65.774118


In [59]:
# Baseline metrics summary
summary = baseline_df.agg({
    "exact_match": "mean",
    "exec_match": "mean",
    "pred_ok": "mean",
    "tokens_per_sec": "mean",
}).to_dict()

print("Baseline metrics (mean over sample):")
for k, v in summary.items():
    if v is None or (isinstance(v, float) and np.isnan(v)):
        print(f"- {k}: n/a")
    else:
        print(f"- {k}: {v:.3f}" if isinstance(v, float) else f"- {k}: {v}")

Baseline metrics (mean over sample):
- exact_match: 0.000
- exec_match: 0.100
- pred_ok: 0.150
- tokens_per_sec: 65.710


---

# Part 4 — QLoRA Training (The Main Event)

We’ll:
1. Load the base model in **4-bit** (GPU recommended)
2. Prepare it for k-bit training
3. Attach LoRA adapters
4. Train with `transformers.Trainer`
5. Save adapters to: `workshop-v1/fico/artifacts/lora_text2sql/`

> CPU is supported for the notebook’s *structure*, but QLoRA training is realistically **GPU-only** (bitsandbytes 4-bit).

In [60]:
# ============================================================
# Build a training dataset in instruction format
# ============================================================

def format_example(ex: Dict[str, Any]) -> Dict[str, Any]:
    prompt = build_prompt(ex["instruction"], ex["context"])
    # Supervised fine-tune text: prompt + gold SQL
    text = prompt + ex["response"].strip()
    return {"text": text}

# Use the stratified split if available (from external dataset cell above)
# Otherwise fall back to the template-based text2sql_data
if 'text2sql_train' in dir() and text2sql_train:
    print(f"Using stratified split: {len(text2sql_train)} train, {len(text2sql_eval)} eval")
    train_ds = Dataset.from_list(text2sql_train).map(format_example)
    eval_ds = Dataset.from_list(text2sql_eval).map(format_example)
else:
    print("Using template-based dataset with random split")
    full_ds = Dataset.from_list(text2sql_data).map(format_example)
    split = full_ds.train_test_split(test_size=0.25, seed=SEED)
    train_ds = split["train"]
    eval_ds = split["test"]

print(f"\nTrain: {train_ds}")
print(f"Eval:  {eval_ds}")
print("\nExample training text:\n")
print(train_ds[0]["text"][:600])

Using stratified split: 198 train, 23 eval


Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]


Train: Dataset({
    features: ['instruction', 'context', 'response', 'difficulty', 'text'],
    num_rows: 198
})
Eval:  Dataset({
    features: ['instruction', 'context', 'response', 'difficulty', 'text'],
    num_rows: 23
})

Example training text:

You are a Text-to-SQL assistant for an SQLite database.
Return ONLY a single SQL query, no explanation.

### Schema
Tables:
- customers (customer_id, name, email, fico_score, created_at)
- accounts (account_id, customer_id, account_type, balance, opened_at)
- transactions (txn_id, account_id, amount, txn_type, txn_date)
- credit_applications (app_id, customer_id, requested_amount, status, decision_date)

### Question
Select the latest 5 credit applications.

### SQL
SELECT * FROM credit_applications ORDER BY app_id DESC LIMIT 5;


In [61]:
# ============================================================
# Tokenization
# ============================================================

MAX_LEN = 512

def tokenize_batch(batch):
    out = tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
    )
    out["labels"] = out["input_ids"].copy()
    return out

train_tok = train_ds.map(tokenize_batch, batched=True, remove_columns=train_ds.column_names)
eval_tok = eval_ds.map(tokenize_batch, batched=True, remove_columns=eval_ds.column_names)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
print("✅ Tokenized")

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

✅ Tokenized


In [62]:
# ============================================================
# Load 4-bit model + attach LoRA adapters (GPU recommended)
# ============================================================

# Adapter save location
ARTIFACT_ROOT = Path("/home/shadeform/workshop-v1/fico/artifacts/lora_text2sql")
ARTIFACT_ROOT.mkdir(parents=True, exist_ok=True)

RUN_NAME = time.strftime("adapter_%Y%m%d_%H%M%S")
ADAPTER_DIR = ARTIFACT_ROOT / RUN_NAME

print("Adapter output dir:", ADAPTER_DIR)

if DEVICE != "cuda":
    print("\n⚠️ Not on GPU. Skipping 4-bit QLoRA model load.")
    print("   You can still read the code; run on a GPU machine to train.")

# Default LoRA hyperparams (widgets below let you change these)
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# Targets for Qwen-style architectures (common)
DEFAULT_TARGET_MODULES = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj",
]

qlora_tokenizer = tokenizer
qlora_model = None

if DEVICE == "cuda":
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    )

    qlora_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
        device_map="auto",
    )

    qlora_model = prepare_model_for_kbit_training(qlora_model)

    lora_cfg = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        target_modules=DEFAULT_TARGET_MODULES,
        bias="none",
        task_type="CAUSAL_LM",
    )

    qlora_model = get_peft_model(qlora_model, lora_cfg)
    qlora_model.print_trainable_parameters()
    print("✅ QLoRA model ready")

Adapter output dir: /home/shadeform/workshop-v1/fico/artifacts/lora_text2sql/adapter_20251215_214938
trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820
✅ QLoRA model ready


### Interactive: Tune LoRA Hyperparameters

Adjust LoRA hyperparameters and training settings. This cell prints:
- Trainable parameters
- A rough adapter size estimate

> For a real run: keep dataset small and training short (~10–20 minutes).

In [63]:
# Widgets for LoRA + training knobs
w_r = widgets.IntSlider(value=16, min=4, max=64, step=4, description="r")
w_alpha = widgets.IntSlider(value=32, min=8, max=256, step=8, description="alpha")
w_dropout = widgets.FloatSlider(value=0.05, min=0.0, max=0.3, step=0.01, description="dropout")

w_epochs = widgets.IntSlider(value=1, min=1, max=5, step=1, description="epochs")
w_train_bs = widgets.IntSlider(value=1, min=1, max=8, step=1, description="batch")
w_grad_accum = widgets.IntSlider(value=8, min=1, max=32, step=1, description="grad_acc")

w_max_steps = widgets.IntSlider(value=120, min=20, max=1000, step=20, description="max_steps")
w_ds_limit = widgets.IntSlider(value=min(64, len(train_tok)), min=16, max=max(16, len(train_tok)), step=16, description="train_n")

w_info = widgets.Output()

def _estimate_adapter_mb(trainable_params: int, dtype_bytes: int = 2) -> float:
    return (trainable_params * dtype_bytes) / 1e6

def refresh_info(*_):
    with w_info:
        clear_output()
        print("Training config preview")
        print("- r:", w_r.value)
        print("- alpha:", w_alpha.value)
        print("- dropout:", w_dropout.value)
        print("- epochs:", w_epochs.value)
        print("- per_device_train_batch_size:", w_train_bs.value)
        print("- gradient_accumulation_steps:", w_grad_accum.value)
        print("- max_steps:", w_max_steps.value)
        print("- train subset size:", w_ds_limit.value)

        if DEVICE != "cuda":
            print("\n⚠️ GPU not detected: QLoRA training cells are for reference.")
            return

        if qlora_model is None:
            print("\nℹ️ QLoRA model object exists only after you run the QLoRA load cell above.")
            return

        trainable = sum(p.numel() for p in qlora_model.parameters() if p.requires_grad)
        print("\nTrainable params:", f"{trainable:,}")
        print("Approx adapter size (fp16/bf16):", f"{_estimate_adapter_mb(trainable):.2f} MB")

for w in [w_r, w_alpha, w_dropout, w_epochs, w_train_bs, w_grad_accum, w_max_steps, w_ds_limit]:
    w.observe(refresh_info, names="value")

display(widgets.VBox([
    widgets.HBox([w_r, w_alpha, w_dropout]),
    widgets.HBox([w_epochs, w_train_bs, w_grad_accum]),
    widgets.HBox([w_max_steps, w_ds_limit]),
    w_info,
]))
refresh_info()

In [64]:
# ============================================================
# Train (QLoRA)
# ============================================================

if DEVICE != "cuda":
    print("⚠️ Skipping training (no GPU).")
else:
    print("Training adapter to:", ADAPTER_DIR)

    # Re-load the quantized base model to apply the current widget settings.
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    )

    qlora_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
        device_map="auto",
    )

    qlora_model.gradient_checkpointing_enable()
    qlora_model = prepare_model_for_kbit_training(qlora_model)

    lora_cfg = LoraConfig(
        r=w_r.value,
        lora_alpha=w_alpha.value,
        lora_dropout=w_dropout.value,
        target_modules=DEFAULT_TARGET_MODULES,
        bias="none",
        task_type="CAUSAL_LM",
    )

    qlora_model = get_peft_model(qlora_model, lora_cfg)
    qlora_model.print_trainable_parameters()

    # Subset training set for speed
    train_subset = train_tok.select(range(min(w_ds_limit.value, len(train_tok))))

    args = TrainingArguments(
        output_dir=str(ADAPTER_DIR / "trainer_out"),
        overwrite_output_dir=True,
        num_train_epochs=w_epochs.value,
        max_steps=w_max_steps.value,
        per_device_train_batch_size=w_train_bs.value,
        gradient_accumulation_steps=w_grad_accum.value,
        learning_rate=2e-4,
        warmup_ratio=0.03,
        logging_steps=10,
        save_strategy="no",
        eval_strategy="no",
        fp16=(not torch.cuda.is_bf16_supported()),
        bf16=torch.cuda.is_bf16_supported(),
        report_to=[],
        optim="paged_adamw_8bit",
    )

    trainer = Trainer(
        model=qlora_model,
        args=args,
        train_dataset=train_subset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    t0 = time.time()
    train_result = trainer.train()
    dt = time.time() - t0
    print(train_result)
    print(f"Train wall time: {dt/60:.1f} min")

    # Save adapter + tokenizer
    ADAPTER_DIR.mkdir(parents=True, exist_ok=True)
    qlora_model.save_pretrained(str(ADAPTER_DIR))
    tokenizer.save_pretrained(str(ADAPTER_DIR))

    # Verify artifacts
    print("\nSaved files:")
    for p in sorted(ADAPTER_DIR.glob("*")):
        print("-", p.name)

    cfg = ADAPTER_DIR / "adapter_config.json"
    if not cfg.exists():
        raise FileNotFoundError(
            f"Training finished but adapter_config.json was not found in {ADAPTER_DIR}. "
            "This usually means save_pretrained did not run or failed."
        )

    print("\n✅ Saved adapter to", ADAPTER_DIR)
    print("✅ Adapter is loadable (adapter_config.json present)")

Training adapter to: /home/shadeform/workshop-v1/fico/artifacts/lora_text2sql/adapter_20251215_214938


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820


Step,Training Loss
10,1.083600
20,0.187500
30,0.140000
40,0.101100
50,0.076400
60,0.056700
70,0.051000
80,0.043700
90,0.036800
100,0.034500


TrainOutput(global_step=120, training_loss=0.1562273661295573, metrics={'train_runtime': 284.3729, 'train_samples_per_second': 3.376, 'train_steps_per_second': 0.422, 'total_flos': 1124647718031360.0, 'train_loss': 0.1562273661295573, 'epoch': 15.0})
Train wall time: 4.7 min

Saved files:
- README.md
- adapter_config.json
- adapter_model.safetensors
- added_tokens.json
- chat_template.jinja
- merges.txt
- special_tokens_map.json
- tokenizer.json
- tokenizer_config.json
- trainer_out
- vocab.json

✅ Saved adapter to /home/shadeform/workshop-v1/fico/artifacts/lora_text2sql/adapter_20251215_214938
✅ Adapter is loadable (adapter_config.json present)


In [65]:
# ============================================================
# Load base + adapter for inference (post-tune) and evaluate
# ============================================================

post_df = None


def _find_latest_adapter_dir(root: Path) -> Path | None:
    if not root.exists():
        return None
    candidates = []
    for p in root.glob("adapter_*/"):
        cfg = p / "adapter_config.json"
        if cfg.exists():
            candidates.append(p)
    if not candidates:
        return None
    return sorted(candidates, key=lambda x: x.name)[-1]


if DEVICE != "cuda":
    print("⚠️ No GPU: skipping adapter eval (you can still run baseline eval above).")
else:
    # Resolve adapter dir robustly (only proceed if adapter_config.json exists)
    artifact_root = Path("/home/shadeform/workshop-v1/fico/artifacts/lora_text2sql")
    adapter_dir = Path(str(ADAPTER_DIR))

    if not (adapter_dir / "adapter_config.json").exists():
        latest = _find_latest_adapter_dir(artifact_root)
        if latest is None:
            print("⚠️ No saved adapter found yet.")
            print("   Run the 'Train (QLoRA)' cell first (it should print '✅ Saved adapter to ...'),")
            print("   then re-run this evaluation cell.")
            print(f"   Expected: {adapter_dir}/adapter_config.json")
            print(f"   Searched: {artifact_root}/adapter_*/adapter_config.json")
        else:
            print(f"⚠️ ADAPTER_DIR has no adapter_config.json. Using latest adapter: {latest}")
            adapter_dir = latest

    if (adapter_dir / "adapter_config.json").exists():
        # Load base model in 4-bit again
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        )

        base4 = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL,
            quantization_config=bnb_config,
            device_map="auto",
        )

        tuned = PeftModel.from_pretrained(base4, str(adapter_dir), is_trainable=False)
        tuned.eval()

        # Use the same eval set as baseline for fair comparison
        if 'text2sql_eval' in dir() and text2sql_eval:
            EVAL_N = min(20, len(text2sql_eval))
            examples_eval = text2sql_eval[:EVAL_N]
            print(f"Evaluating on {EVAL_N} held-out examples")
        else:
            EVAL_N = 8
            examples_eval = text2sql_data[:EVAL_N]
        
        preds = []
        lats = []
        for ex in examples_eval:
            pred, lat = generate_sql(tuned, tokenizer, ex["instruction"], ex["context"], max_new_tokens=128)
            preds.append(pred)
            lats.append(lat)

        post_df = evaluate_predictions(conn, examples_eval, preds)
        post_df["tokens_per_sec"] = [x.get("tokens_per_sec") for x in lats]
        post_df

Evaluating on 20 held-out examples


### 📊 Comparison: Baseline vs Fine-Tuned

Run this cell after both the baseline evaluation (Part 3) and post-tune evaluation above to see a side-by-side comparison of performance metrics.

In [68]:
# ============================================================
# Compare Baseline vs Fine-Tuned Model
# ============================================================

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Check that both dataframes exist
if 'baseline_df' not in dir() or baseline_df is None:
    print("⚠️ Run the baseline evaluation cell first (Part 3).")
elif 'post_df' not in dir() or post_df is None:
    print("⚠️ Run the post-tune evaluation cell first (after training).")
else:
    # Compute summary metrics for both
    metrics = ["exact_match", "exec_match", "result_match"]
    
    baseline_means = {m: baseline_df[m].mean() for m in metrics if m in baseline_df.columns}
    posttune_means = {m: post_df[m].mean() for m in metrics if m in post_df.columns}
    
    # Create comparison dataframe
    comparison = pd.DataFrame({
        "Metric": list(baseline_means.keys()),
        "Baseline (Before)": [baseline_means[m] * 100 for m in baseline_means.keys()],
        "Fine-Tuned (After)": [posttune_means.get(m, 0) * 100 for m in baseline_means.keys()],
    })
    comparison["Improvement"] = comparison["Fine-Tuned (After)"] - comparison["Baseline (Before)"]
    comparison["Improvement (%)"] = (comparison["Improvement"] / comparison["Baseline (Before)"].replace(0, 1)) * 100
    
    print("=" * 60)
    print("📊 COMPARISON: Baseline vs Fine-Tuned Model")
    print("=" * 60)
    print(f"\nSamples evaluated: Baseline={len(baseline_df)}, Fine-Tuned={len(post_df)}")
    print("\n" + comparison.to_string(index=False, float_format=lambda x: f"{x:.1f}%"))
    
    # Latency comparison if available
    if "tokens_per_sec" in baseline_df.columns and "tokens_per_sec" in post_df.columns:
        base_tps = baseline_df["tokens_per_sec"].mean()
        post_tps = post_df["tokens_per_sec"].mean()
        print(f"\n⚡ Throughput: Baseline={base_tps:.1f} tok/s, Fine-Tuned={post_tps:.1f} tok/s")
    
    # Create visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Accuracy Metrics (%)", "Per-Sample Comparison"),
        specs=[[{"type": "bar"}, {"type": "scatter"}]]
    )
    
    # Bar chart comparing metrics
    x_labels = comparison["Metric"].tolist()
    fig.add_trace(
        go.Bar(name="Baseline", x=x_labels, y=comparison["Baseline (Before)"].tolist(), 
               marker_color="lightcoral", text=[f"{v:.1f}%" for v in comparison["Baseline (Before)"]], textposition="outside"),
        row=1, col=1
    )
    fig.add_trace(
        go.Bar(name="Fine-Tuned", x=x_labels, y=comparison["Fine-Tuned (After)"].tolist(), 
               marker_color="mediumseagreen", text=[f"{v:.1f}%" for v in comparison["Fine-Tuned (After)"]], textposition="outside"),
        row=1, col=1
    )
    
    # Scatter plot: per-sample exec_match comparison (if same samples)
    if len(baseline_df) == len(post_df) and "exec_match" in baseline_df.columns:
        # Jitter for visibility
        import numpy as np
        jitter = np.random.uniform(-0.05, 0.05, len(baseline_df))
        fig.add_trace(
            go.Scatter(
                x=baseline_df["exec_match"] + jitter, 
                y=post_df["exec_match"] + jitter,
                mode="markers",
                marker=dict(size=10, opacity=0.6, color="steelblue"),
                name="Samples",
                hovertemplate="Baseline: %{x}<br>Fine-Tuned: %{y}<extra></extra>"
            ),
            row=1, col=2
        )
        # Diagonal line (no improvement)
        fig.add_trace(
            go.Scatter(x=[0, 1], y=[0, 1], mode="lines", line=dict(dash="dash", color="gray"), 
                       name="No Change", showlegend=False),
            row=1, col=2
        )
        fig.update_xaxes(title_text="Baseline exec_match", range=[-0.1, 1.1], row=1, col=2)
        fig.update_yaxes(title_text="Fine-Tuned exec_match", range=[-0.1, 1.1], row=1, col=2)
    
    fig.update_layout(
        title="🔬 Model Comparison: Before vs After Fine-Tuning",
        barmode="group",
        height=450,
        showlegend=True
    )
    fig.update_yaxes(range=[0, 110], row=1, col=1)
    fig.show()
    
    # Show examples where fine-tuning helped
    if "exec_match" in baseline_df.columns and "exec_match" in post_df.columns:
        improved = (post_df["exec_match"] == 1) & (baseline_df["exec_match"] == 0)
        regressed = (post_df["exec_match"] == 0) & (baseline_df["exec_match"] == 1)
        
        print(f"\n✅ Queries FIXED by fine-tuning: {improved.sum()}")
        print(f"❌ Queries REGRESSED by fine-tuning: {regressed.sum()}")
        
        if improved.sum() > 0:
            print("\n--- Example of FIXED query ---")
            idx = improved.idxmax()
            print(f"Question: {baseline_df.loc[idx, 'instruction']}")
            print(f"Baseline pred: {baseline_df.loc[idx, 'pred_sql']}")
            print(f"Fine-tuned pred: {post_df.loc[idx, 'pred_sql']}")
            print(f"Gold SQL: {baseline_df.loc[idx, 'gold_sql']}")

📊 COMPARISON: Baseline vs Fine-Tuned Model

Samples evaluated: Baseline=20, Fine-Tuned=20

     Metric  Baseline (Before)  Fine-Tuned (After)  Improvement  Improvement (%)
exact_match               0.0%               40.0%        40.0%          4000.0%
 exec_match              10.0%               60.0%        50.0%           500.0%

⚡ Throughput: Baseline=65.7 tok/s, Fine-Tuned=17.6 tok/s



✅ Queries FIXED by fine-tuning: 10
❌ Queries REGRESSED by fine-tuning: 0

--- Example of FIXED query ---
Question: Show the latest transaction date for each account.
Baseline pred: ```sql
Fine-tuned pred: SELECT account_id, MAX(txn_date) FROM transactions GROUP BY account_id;
Gold SQL: SELECT account_id, MAX(txn_date) FROM transactions GROUP BY account_id;


In [67]:
# Load base + adapter for inference (deployment pattern)
if DEVICE != "cuda":
    print("CPU mode: shown for reference")
else:
    artifact_root = Path("/home/shadeform/workshop-v1/fico/artifacts/lora_text2sql")
    adapter_dir = Path(str(ADAPTER_DIR))
    if not (adapter_dir / "adapter_config.json").exists():
        adapter_dir = _find_latest_adapter_dir(artifact_root) or adapter_dir

    if not (adapter_dir / "adapter_config.json").exists():
        print("⚠️ No saved adapter found. Run the 'Train (QLoRA)' cell first.")
    else:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        )

        base4 = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL,
            quantization_config=bnb_config,
            device_map="auto",
        )

        deployed = PeftModel.from_pretrained(base4, str(adapter_dir), is_trainable=False)
        deployed.eval()

        # Quick demo
        ex = text2sql_data[0]
        pred, lat = generate_sql(deployed, tokenizer, ex["instruction"], ex["context"], max_new_tokens=128)
        print("Question:", ex["instruction"])
        print("Gold SQL:", ex["response"])
        print("Pred SQL:", pred)
        print("Latency:", lat)

Question: Select the latest 5 credit applications.
Gold SQL: SELECT * FROM credit_applications ORDER BY app_id DESC LIMIT 5;
Pred SQL: SELECT * FROM credit_applications ORDER BY app_id DESC LIMIT 5;
Latency: {'seconds': 7.46020770072937, 'new_tokens': 128, 'tokens_per_sec': 17.157699240395907}
